JDE
$$
F_{i,G+1}=
\begin{cases}
F_l+rand_1*F_u, &if\ rand_2\lt \tau_1\\
F_{i,G}, &otherwise
\end{cases}
$$
$$
CR_{i,G+1}=
\begin{cases}
rand_3, &if\ rand_4\lt \tau_2\\
CR_{i,G}, &otherwise
\end{cases}
$$
$$rand_j\in[0,1],\tau_1=\tau_2=0.1,F_l=0.1,F_u=0.9$$

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
def jde(fobj, bounds, mut=0.5, crossp=0.9, popsize=20, its=1000, goal = 0):
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    pop_denorm = min_b + pop * diff
    fitness = np.asarray([fobj(ind) for ind in pop_denorm])
    best_idx = np.argmin(fitness)
    best = pop_denorm[best_idx]
    for i in range(its):
        for j in range(popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace = False)]
            mutant = np.clip(a + mut * (b - c), 0, 1)
            cross_points = np.random.rand(dimensions) < crossp
            randj = np.random.rand(4)
            if randj[0]<0.1:
                mut = 0.1+randj[1]*0.9
            if randj[2]<0.1:
                crossp = randj[3]
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, pop[j])
            trial_denorm = min_b + trial * diff
            f = fobj(trial_denorm)
            if f < fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm  
        if(np.fabs(min(fitness) - goal) < 1e-6):
            break
        yield best, fitness[best_idx]

SaDE策略池中四个策略,rand_1,rand_to_best_2,rand_2,current_to_rand_1
开始每个策略被选择的概率相等,在每一代对于每个策略都考虑使用这个策略生成的变异向量,这些变异向量进入下一次迭代的数量记为$ns_{k,G}$,反之记为$nf_{k,G}$,这样记录LP代，LP称为学习期。LP溢出时就删除最早的记录，将最新生成的数字再填写进去。于是再最初的LP代之后将基于成功和失败记忆在之后的每代更新选择不同策略的概率。

首先在前LP代，对于NP个个体其中一个，生成一个随机数，由随机数确定采用什么变异策略（每种策略被选择的概率暂时相等）。然后根据选择的策略执行相应的变异函数（选择几个其他个体或最优个体进行变异操作）得到变异个体，之后交叉，判断能否进入下一次迭代，记录进成功或失败表格。
在LP代之后，选择某个变异策略的概率每代都会变化，这个变化根据成功和失败的两个表格得出。（表格记得清零），然后更新表格。（取余LP）

In [ ]:
import numpy as np
def sade(fobj, bounds, mut=0.5, crossp=0.9, popsize=20, its=1000, goal = 0):
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    pop_denorm = min_b + pop * diff
    fitness = np.asarray([fobj(ind) for ind in pop_denorm])
    best_idx = np.argmin(fitness)
    best = pop_denorm[best_idx]
    sp = strategy_probability = [0.25, 0.25, 0.25, 0.25]
    lp = learning_period = 5
    success_memory = failure_memory = np.zeros([lp, 4])
    for i in range (lp):
        for j in range (popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c, d, e = pop[np.random.choice(idxs, 5, replace = False)]
            rand_sp = np.random.rand()
            memory_colume = -1
            if(rand_sp < 0.25):
                memory_colume = 0
                trial = rand_1_bin(a, b, c, mut)
            elif(rand_sp < 0.5):
                memory_colume = 1
                trial = rand_to_best_2_bin()
            elif(rand_sp < 0.75):
                memory_colume = 2
                trial = rand_2_bin()
            else:
                memory_colume = 3
                trial = current_to_rand_1_bin()
            trial_denorm = min_b + trial * diff
            f = fobj(trial_denorm)
            if f < fitness[j]:
                fitness[j] = f
                pop[j] = trial
                success_memory[i, memory_colume] += 1
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm
            else:
                failure_memory[i, memory_colume] += 1

In [ ]:
def rand_1_bin(a, b, c, mut):
    mutant = np.clip(a + mut * (b - c), 0, 1)
    cross_points = np.random.rand(dimensions) < crossp
    if not np.any(cross_points):
        cross_points[np.random.randint(0, dimensions)] = True
    trial = np.where(cross_points, mutant, pop[j])
    return trial
def rand_to_best_2_bin()

In [6]:
def Rastrigin(x):
    return sum(x ** 2 - 10 * np.cos(2 * np.pi * x) + 10)
def Rastrigin_jde():
    start = datetime.datetime.now()
    it = list(jde(Rastrigin, [(-5.12,5.12)] * 30, mut = 0.5, crossp = 0.9, popsize = 100, its = 2000))
    print(it[-1])
    x, f = zip(*it)
    plt.plot(f, label='Rastrigin with jde')
    plt.yscale('log')
    plt.legend()
    plt.savefig('Rastrigin with jde')
    end = datetime.datetime.now()
    print(end - start)

In [ ]:
def jde(fobj, bounds, mut=0.5, crossp=0.9, popsize=20, its=1000, goal = 0):
    dimensions = len(bounds)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    pop_denorm = min_b + pop * diff
    fitness = np.asarray([fobj(ind) for ind in pop_denorm])
    best_idx = np.argmin(fitness)
    best = pop_denorm[best_idx]
    for i in range(its):
        for j in range(popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace = False)]
            mutant = np.clip(a + mut * (b - c), 0, 1)
            cross_points = np.random.rand(dimensions) < crossp
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, pop[j])
            trial_denorm = min_b + trial * diff
            f = fobj(trial_denorm)
            if f < fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm  
        if(np.fabs(min(fitness) - goal) < 1e-6):
            break
        yield best, fitness[best_idx]